# Ambulance service sanity checks

The LSOA to ambulance service lookup file is quite large. Can we reduce the size considerably by using a CCG 2015 to ambulance lookup that gives the exact same results?

## Summary

When creating the LSOA-ambulance service lookup previously, we mostly used a CCG 2021 - ambulance service lookup table and added in a few special cases. The special cases were sorted out on the LSOA-level by using the LSOA names or the Local Authority Districts that contained them.

It would be easier to wrangle the data if the ambulance services across all of England and Wales could be defined not at the LSOA level but by using larger areas.

The CCG 2021 - ambulance lookup file uses the CCGs from 2021. There are about 100 of these CCGs. Back in 2015, there were about twice as many CCGs that were later mostly merged into the fewer bigger CCGs in the 2021 data. Perhaps the special cases in the lookup file are due to these merged CCGs, and perhaps if we look back to before the merges then we can find a more straightforward lookup between 2015 CCGs and ambulance services.

The following figure shows how the CCGs have changed from 2015 to 2021. In 2015, the Isle of Wight had its own CCG. By 2021 it had been merged into Hampshire.

![The CCGs in Hampshire in the Isle of Wight. One subplot shows the CCGs from 2015, the other shows the CCGs from 2021.](figures/hants_ccg.png)

And the following figure picks out just Milton Keynes. In 2015 it was its own CCG. In 2021 it is included in the combined "NHS Bedfordshire, Luton and Milton Keynes CCG", which is now split across two ambulance service regions.

![The Milton Keynes CCG shown in separate maps as either part of the "NHS Bedfordshire, Luton and Milton Keynes CCG" or as part of the "South Central Ambulance Service" region.](figures/ambo_milton_keynes.png)

## Aim

To create a lookup table of CCG 2015 to ambulance service region with no special cases.

## Notebook setup

In [1]:
# Import required packages
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    data = './data'
    look_ups = 'look_ups'

    lsoa_to_ccg15 = ('Lower_Layer_Super_Output_Area_(2011)_to_' +
                     'Clinical_Commissioning_Group_to_' +
                     'Local_Authority_District_(July_2015)_Lookup_in_England.csv')

    lsoa_ambo_results = 'lsoa_ambo_lookup.csv'
    
    ccg15_amb_lookup = 'ccg15_amb.csv'


paths = Paths()

## LSOA ambulance boundaries

In [3]:
lsoa_amb_data = pd.read_csv(os.path.join(paths.data, paths.lsoa_ambo_results))

lsoa_amb_data.head()

,LSOA11NM,LSOA11CD,ambo21,ambo22
0,Adur 001A,E01031349,SECAmb,SECAmb
1,Adur 001B,E01031350,SECAmb,SECAmb
2,Adur 001C,E01031351,SECAmb,SECAmb
3,Adur 001D,E01031352,SECAmb,SECAmb
4,Adur 001E,E01031370,SECAmb,SECAmb


## Compare catchment areas with CCG 2015

Import LSOA-CCG15 lookup:

In [4]:
df_lsoa_ccg15 = pd.read_csv(os.path.join(paths.data, paths.look_ups, paths.lsoa_to_ccg15))

In [5]:
df_lsoa_ccg15.head()

,LSOA11CD,LSOA11NM,CCG15CD,CCG15CDH,CCG15NM,LAD15CD,LAD15NM,FID
0,E01010638,Bradford 004A,E38000001,02N,"NHS Airedale, Wharfedale and Craven CCG",E08000032,Bradford,1
1,E01023975,Ashford 007B,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,2
2,E01010639,Bradford 004B,E38000001,02N,"NHS Airedale, Wharfedale and Craven CCG",E08000032,Bradford,3
3,E01023976,Ashford 008A,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,4
4,E01023977,Ashford 007C,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,5


Import a CCG15-ambulance service lookup that we've invented:

In [6]:
# Build ambulance service shapes from CCG 2015:
df_ccg15_ambo = pd.read_csv(os.path.join(paths.data, paths.ccg15_amb_lookup))

df_ccg15_ambo

,ccg15nm,ambo21
0,"NHS Airedale, Wharfedale and Craven CCG",YAS
1,NHS Ashford CCG,SECAmb
2,NHS Aylesbury Vale CCG,SCAS
3,NHS Barking and Dagenham CCG,LAS
4,NHS Barnet CCG,LAS
...,...,...
204,"NHS Windsor, Ascot and Maidenhead CCG",SECAmb
205,NHS Wirral CCG,NWAS
206,NHS Wokingham CCG,SCAS
207,NHS Wolverhampton CCG,WMAS


Create a dataframe containing all LSOA and the ambulance service from the CCG2015 lookup:

In [7]:
df_lsoa_ccg15_ambo = pd.merge(
    df_lsoa_ccg15, df_ccg15_ambo,
    left_on='CCG15NM', right_on='ccg15nm', how='left'
)

Create a dataframe containing all LSOA, the ambulance service data from the LSOA, and the ambulance service data that we've just created using CCG 2015.

In [11]:
lsoa_amb_data = lsoa_amb_data.rename(
    columns={'ambo21': 'ambo_from_lsoa11'})
df_lsoa_ccg15_ambo = df_lsoa_ccg15_ambo.rename(
    columns={'ambo21': 'ambo_from_ccg15'})

df_lsoa_compare = pd.merge(
    lsoa_amb_data[['LSOA11NM', 'LSOA11CD', 'ambo_from_lsoa11']],
    df_lsoa_ccg15_ambo[['LSOA11CD', 'ambo_from_ccg15']],
    on='LSOA11CD', how='right'
)

In [12]:
df_lsoa_compare.head()

,LSOA11NM,LSOA11CD,ambo_from_lsoa11,ambo_from_ccg15
0,Bradford 004A,E01010638,YAS,YAS
1,Ashford 007B,E01023975,SECAmb,SECAmb
2,Bradford 004B,E01010639,YAS,YAS
3,Ashford 008A,E01023976,SECAmb,SECAmb
4,Ashford 007C,E01023977,SECAmb,SECAmb


Do the ambulance service columns differ for any LSOA?

In [13]:
mask = df_lsoa_compare['ambo_from_lsoa11'] == df_lsoa_compare['ambo_from_ccg15']

In [14]:
mask.all()

True

The two columns are always the same, so we can use CCG 2015 to define the 2021 ambulance service boundaries!